In [1]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import scipy

In [2]:
def read_coords():
    coords_dict = {}
    model_count = 0
    with open("1fsd.pdb",'r') as f:
        for line in f:
            tmp_line = line.split()
            if tmp_line[0] == "MODEL":
                model_count += 1
                coords_dict[model_count] = []

            elif tmp_line[0]  == 'ATOM' and tmp_line[2] == 'CA':
                coords = tmp_line[6:9]
                coords_dict[model_count].append(coords)

    return coords_dict

In [3]:
# eigendecomposition
def evd(matrix):
    eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(matrix, k=5)
    lambda_matrix = np.diag(eigenvalues)
    return lambda_matrix, eigenvectors

In [4]:
# восстановление первоначальной матрицы порядка 28
def compose_matrix(eigenvectors, eigenvalues):

    return (eigenvectors @ eigenvalues @ eigenvectors.transpose())

In [5]:
def is_same_matrix(matrix, composed_matrix):
    return np.allclose(matrix, composed_matrix)

In [6]:
def compute_model(model_coords):

    matrix = euclidean_distances(model_coords, model_coords) ** 2
    print('rank =', np.linalg.matrix_rank(matrix))
        
    eigenvalues, eigenvectors = evd(matrix)
    # матрица eigenvalues имеет меньшую размерность
    print('shape of new matrix = ', eigenvalues.shape)

    composed_matrix = compose_matrix(eigenvectors, eigenvalues)
    print('Are matrix and composed matrix the same?', is_same_matrix(matrix, composed_matrix))

In [7]:
# Task №3
# Координаты CA находятся в общем положении (general position) -> 
# -> rank EDM (матрицы квадратов попарных расстояний) = 5 = min(28, 5) = min(28, d+2), где d = 3 

In [8]:
coords_dict = read_coords()
for model_id in range(1, 42):
    print('MODEL', model_id, end=', ')
    compute_model(coords_dict[model_id])

MODEL 1, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 2, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 3, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 4, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 5, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 6, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 7, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 8, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 9, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 10, rank = 5
shape of new matrix =  (5, 5)
Are matrix and composed matrix the same? True
MODEL 11, rank = 5
shape of new matrix =  (5, 5)
Are matrix

In [9]:
# В ходе рассмотреняи факторизаций также было протестировано сингулярное разложение матрицы (SVD), 
# также позволяющее получить из изначальной матрицы EDM матрицу меньшей размерности, а именно матрицу порядка 5.
# Однако SVD для EDM проигрывает в скорости.

In [7]:
Вариант получения только одной(!) матрицы(но размером (5, 28)), из изначальной матрицы M(28, 28):
1) находится rank(M)=5 линейно независимых строк
2) эти линейно независимые строки дадут нам rank(M) столбцов, так как матрица симметрична
3) для каждой из линейно зависимых строк можно составить систему линейных уравнений вида 
(Не умоляя общности, предположим, что выбранные линейно незавсимые строки являются первыми rank(M) строками матрицы)
a11*x1 + a21*x2 + a31*x3 + a41*x4 + a51 *x5 = a16 (так как матрица симметрична) 
a12*x1 + a22*x2 + a32*x3 + a42*x4 + a52 *x5 = a26 
a13*x1 + a23*x2 + a33*x3 + a43*x4 + a54 *x5 = a36 
a14*x1 + a24*x2 + a34*x3 + a44*x4 + a54 *x5 = a46 
a15*x1 + a25*x2 + a35*x3 + a45*x4 + a55 *x5 = a56 

где aij - элемент матрицы М, стоящей в i-той строке, j-том столбце

Таким образом, у нас получилась система из 5 линейных уравнений с 5 неизвестными.
Решив ее для линейной зависмой строки, мы восстановим коэффиценты для линейной комбинации линейно независимых строк.
С помощью этих коэффциентов восстанавливаются все элементы строки.
Проделав это для всех линейно зависимых строк, найдем всю матрицу.

### 